In [2]:
import pandas as pd

# Load the dataset and rename the original index column to "original index"
df = pd.read_csv('../data/intern_screening_dataset.csv').reset_index().rename(columns={'index': 'original_index'})
df.head()

,original_index,question,answer
0,0,What is (are) Glaucoma ?,Glaucoma is a group of diseases that can damag...
1,1,What is (are) Glaucoma ?,The optic nerve is a bundle of more than 1 mil...
2,2,What is (are) Glaucoma ?,Open-angle glaucoma is the most common form of...
3,3,Who is at risk for Glaucoma? ?,Anyone can develop glaucoma. Some people are a...
4,4,How to prevent Glaucoma ?,"At this time, we do not know how to prevent gl..."


In [3]:
# Check the columns and data types
print("Columns:", list(df.columns))
print("Data types:", [str(dt) for dt in df.dtypes])
print("Count:", df['question'].count())
print("Missing values in question:", df['question'].isnull().sum())
print("Missing values in answer:", df['answer'].isnull().sum())
print("Unique values in question:", df['question'].nunique())
print("Unique values in answer:", df['answer'].nunique())

Columns: ['original_index', 'question', 'answer']
Data types: ['int64', 'object', 'object']
Count: 16406
Missing values in question: 0
Missing values in answer: 5
Unique values in question: 14981
Unique values in answer: 15811


In [4]:
# Calculate statistics about the length of the answers
answer_lengths = df['answer'].apply(lambda x: "" if pd.isna(x) else x).apply(len)
print("Answer length statistics:")
print(f"  Max length: {answer_lengths.max()}")
print(f"  Min length: {answer_lengths.min()}")
print(f"  Mean length: {answer_lengths.mean():.2f}")
print(f"  Median length: {answer_lengths.median()}")
print(f"  Standard deviation: {answer_lengths.std():.2f}")

Answer length statistics:
  Max length: 29046
  Min length: 0
  Mean length: 1302.61
  Median length: 889.0
  Standard deviation: 1656.13


In [5]:
# Remove rows where 'answer' is missing
df_clean = df.dropna(subset=['answer'])

# For each question, concatenate all answers and collect the list of original indices
df_concat = df_clean.groupby('question').agg({
    'answer': lambda x: '\n---\n'.join(x),
    'original_index': lambda x: list(x)
}).reset_index()

# Create a new column 'question_index' with the index of each unique question
df_concat['question_index'] = df_concat.index

df_concat.head()

,question,answer,original_index,question_index
0,Do you have information about A1C,Summary : A1C is a blood test for type 2 diabe...,[1909],0
1,Do you have information about Acupuncture,Summary : Acupuncture has been practiced in Ch...,[1744],1
2,Do you have information about Adoption,Summary : Adoption brings a child born to othe...,[2307],2
3,Do you have information about Advance Directives,Summary : What kind of medical care would you ...,[1937],3
4,Do you have information about African American...,Summary : Every racial or ethnic group has spe...,[2434],4


In [6]:
# Check the columns and data types
print("Columns:", list(df_concat.columns))
print("Data types:", [str(dt) for dt in df_concat.dtypes])
print("Count:", df_concat['question'].count())
print("Missing values in question:", df_concat['question'].isnull().sum())
print("Missing values in answer:", df_concat['answer'].isnull().sum())
print("Unique values in question:", df_concat['question'].nunique())
print("Unique values in answer:", df_concat['answer'].nunique())

Columns: ['question', 'answer', 'original_index', 'question_index']
Data types: ['object', 'object', 'object', 'int64']
Count: 14976
Missing values in question: 0
Missing values in answer: 0
Unique values in question: 14976
Unique values in answer: 14470


In [7]:
import json

# Save a mapping from question_index to question and original_index
question_map = df_concat[['question_index', 'question', 'original_index']].set_index('question_index').to_dict(orient='index')
with open('../data/question_index_map.json', 'w', encoding='utf-8') as f:
    json.dump(question_map, f, ensure_ascii=False, indent=2)

In [8]:
# Group by 'answer' to aggregate duplicated answers
df_answer_grouped = df_concat.groupby('answer').agg({
    # Concatenate all unique questions into a single string, separated by '\n---\n'
    'question': lambda x: '\n---\n'.join(sorted(set(x))),
    # Concatenate all lists of original_index into a single list
    'original_index': lambda x: sum(x, []),
    # Collect all question_index values into a list
    'question_index': lambda x: list(x)
}).reset_index()

# Create a new column 'answer_index' as a unique index for each answer
df_answer_grouped['answer_index'] = df_answer_grouped.index

df_answer_grouped.head()


,answer,question,original_index,question_index,answer_index
0,- A kidney stone is a solid piece of material ...,What to do for Kidney Stones in Adults ?,[15613],[14255],0
1,- A person may prevent or delay some health pr...,What to do for Nutrition for Advanced Chronic ...,[15567],[14268],1
2,- Acromegaly is a hormonal disorder that resul...,What to do for Acromegaly ?,[15719],[14196],2
3,- Bladder problems have many possible causes. ...,What to do for What I need to know about Inter...,[16186],[14315],3
4,- Cirrhosis is scarring of the liver. Scar tis...,What to do for What I need to know about Cirrh...,[15262],[14304],4


In [9]:
import json

# Save a mapping from question_index to question and original_index
question_map = df_answer_grouped[['answer_index', 'answer', 'original_index']].set_index('answer_index').to_dict(orient='index')
with open('../data/answer_index_map.json', 'w', encoding='utf-8') as f:
    json.dump(question_map, f, ensure_ascii=False, indent=2)

In [10]:
# Calculate statistics about the length of the answers
answer_lengths = df_answer_grouped['answer'].apply(len)
print("Answer length statistics:")
print(f"  Max length: {answer_lengths.max()}")
print(f"  Min length: {answer_lengths.min()}")
print(f"  Mean length: {answer_lengths.mean():.2f}")
print(f"  Median length: {answer_lengths.median()}")
print(f"  Standard deviation: {answer_lengths.std():.2f}")


Answer length statistics:
  Max length: 117627
  Min length: 6
  Mean length: 1461.92
  Median length: 957.0
  Standard deviation: 2200.87


In [ ]:
import nltk
from transformers import AutoTokenizer

try:
    nltk.data.find('tokenizers/punkt')
except Exception:
    print("Downloading the 'punkt' sentence tokenizer from NLTK...")
    nltk.download('punkt')

try:
    nltk.data.find('tokenizers/punkt_tab')
except Exception:
    print("Downloading the 'punkt_tab' sentence tokenizer from NLTK...")
    nltk.download('punkt_tab')



def chunk_text(text: str, tokenizer, chunk_size: int = 400, chunk_overlap: int = 50) -> list[str]:
    """
    Divide a long text into smaller chunks, respecting the sentence boundaries
    and with a defined overlap in number of tokens.

    Args:
        text (str): The complete text to be divided.
        tokenizer: The tokenizer from the Transformers library to be used to count the tokens.
        chunk_size (int): The maximum size of each chunk in tokens.
        chunk_overlap (int): The number of tokens of overlap between consecutive chunks.

    Returns:
        list[str]: A list of strings, where each string is a chunk of text.
    """
    if not text:
        return []

    # 1. Dividir o texto em sentenças usando NLTK
    sentences = nltk.sent_tokenize(text)

    # 2. Tokenizar todas as sentenças e guardar seus tokens
    tokens = [tokenizer.encode(sentence, add_special_tokens=False) for sentence in sentences]
    
    chunks = []
    current_chunk_tokens = []
    current_chunk_sentences = []
    
    for i, sentence_tokens in enumerate(tokens):
        # Se adicionar a próxima sentença ultrapassar o tamanho do chunk
        if len(current_chunk_tokens) + len(sentence_tokens) > chunk_size:
            # Finaliza o chunk atual e adiciona à lista
            if current_chunk_tokens:
                chunk_str = tokenizer.decode(current_chunk_tokens)
                chunks.append(chunk_str.strip())
            
            # 3. Começa um novo chunk com sobreposição
            # Pega os últimos tokens do chunk que acabamos de criar para formar a sobreposição
            overlap_tokens = current_chunk_tokens[-chunk_overlap:] if chunk_overlap > 0 and current_chunk_tokens else []
            current_chunk_tokens = overlap_tokens + sentence_tokens
        else:
            current_chunk_tokens.extend(sentence_tokens)

    # Adiciona o último chunk que sobrou
    if current_chunk_tokens:
        chunk_str = tokenizer.decode(current_chunk_tokens)
        chunks.append(chunk_str.strip())
        
    return chunks

[nltk_data] Downloading package punkt_tab to C:\Users\Aron
[nltk_data]     Ifanger\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt_tab.zip.


In [14]:
# Load the same tokenizer that you will use in your BERT/DistilBERT model
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')

df_answer_grouped['answer_chunks'] = df_answer_grouped['answer'].apply(
    lambda x: chunk_text(x, tokenizer, chunk_size=100, chunk_overlap=20)
)

df_chunks = df_answer_grouped.explode('answer_chunks')
df_chunks["chunk_index"] = df_chunks.index

df_chunks.head()

Token indices sequence length is longer than the specified maximum sequence length for this model (574 > 512). Running this sequence through the model will result in indexing errors


,answer,question,original_index,question_index,answer_index,answer_chunks,chunk_index
0,- A kidney stone is a solid piece of material ...,What to do for Kidney Stones in Adults ?,[15613],[14255],0,- a kidney stone is a solid piece of material ...,0
0,- A kidney stone is a solid piece of material ...,What to do for Kidney Stones in Adults ?,[15613],[14255],0,scientists do not believe that eating any spec...,0
0,- A kidney stone is a solid piece of material ...,What to do for Kidney Stones in Adults ?,[15613],[14255],0,"##gnose kidney stones, the health care provide...",0
0,- A kidney stone is a solid piece of material ...,What to do for Kidney Stones in Adults ?,[15613],[14255],0,", as well as whether they are causing pain or ...",0
1,- A person may prevent or delay some health pr...,What to do for Nutrition for Advanced Chronic ...,[15567],[14268],1,- a person may prevent or delay some health pr...,1


In [15]:
# Calculate statistics about the length of the answers
chunks_lengths = df_chunks['answer_chunks'].apply(len)
print("Answer length statistics:")
print(f"  Max length: {chunks_lengths.max()}")
print(f"  Min length: {chunks_lengths.min()}")
print(f"  Mean length: {chunks_lengths.mean():.2f}")
print(f"  Median length: {chunks_lengths.median()}")
print(f"  Standard deviation: {chunks_lengths.std():.2f}")

Answer length statistics:
  Max length: 5926
  Min length: 4
  Mean length: 398.75
  Median length: 392.0
  Standard deviation: 159.81


In [16]:
# Save the df_chunks DataFrame to a CSV file for later use
df_chunks.to_csv("../data/intern_screening_dataset_chunks.csv", index=False)

In [42]:
from tqdm import tqdm

def chunk_text_by_sentence(text: str, tokenizer, chunk_size: int = 400, overlap_sentences: int = 1) -> list[str]:
    """
    Splits a text into chunks, with overlap based on whole sentences.
    This is a more robust approach to ensure text integrity.
    """
    if not isinstance(text, str) or not text:
        return []

    # 1. Split the text into sentences
    sentences = nltk.sent_tokenize(text)
    
    # 2. Group sentences into chunks
    chunks = []
    current_chunk_sentences = []
    current_chunk_tokens = 0

    for sentence in sentences:
        sentence_tokens = tokenizer.tokenize(sentence)
        
        # If adding the next sentence exceeds the limit
        if current_chunk_tokens + len(sentence_tokens) > chunk_size and current_chunk_sentences:
            # Finalize the current chunk
            chunks.append(" ".join(current_chunk_sentences))
            
            # Start a new chunk with sentence overlap
            current_chunk_sentences = current_chunk_sentences[-overlap_sentences:]
            current_chunk_tokens = len(tokenizer.tokenize(" ".join(current_chunk_sentences)))
        
        current_chunk_sentences.append(sentence)
        current_chunk_tokens += len(sentence_tokens)

    # Add the last chunk
    if current_chunk_sentences:
        chunks.append(" ".join(current_chunk_sentences))
        
    return chunks

# --- PREPARATION SCRIPT ---

def create_squad_dataset_pipeline(df_original: pd.DataFrame, tokenizer, max_answer_len_tokens: int = 512):
    """
    Complete pipeline to create the knowledge base (chunks) and SQuAD training data.
    """
    # 1. Create the knowledge base (KB) with the new chunking function
    print("Step 1: Aggregating answers to create long contexts...")
    df_contexts = df_original.groupby('question')['answer'].apply(lambda x: ' '.join(str(i) for i in x)).reset_index()
    df_contexts.columns = ['topic', 'long_context']

    print("Step 2: Applying sentence-based chunking to long contexts...")
    all_chunks = []
    for context in tqdm(df_contexts['long_context']):
        chunks = chunk_text_by_sentence(context, tokenizer, chunk_size=400, overlap_sentences=1)
        all_chunks.extend(chunks)

    unique_chunks = list(set(all_chunks))
    print(f"Knowledge base created with {len(unique_chunks)} unique chunks.")

    # 2. Create the SQuAD training dataset
    training_data = []
    print(f"\nStep 3: Generating training data from {len(df_original)} original rows...")
    
    # Counter for debugging
    found_in_chunk_count = 0

    for _, row in tqdm(df_original.iterrows(), total=df_original.shape[0]):
        question = str(row['question'])
        answer_text = str(row['answer'])
        answer_tokens_len = len(tokenizer.tokenize(answer_text))

        if 0 < answer_tokens_len < max_answer_len_tokens:
            # --- SCENARIO A: Short Answer ---
            for chunk_context in unique_chunks:
                # The 'in' search is now much more likely to work
                if answer_text in chunk_context:
                    start_index = chunk_context.find(answer_text)
                    training_data.append({
                        'context': chunk_context, 'question': question,
                        'answers': {'text': [answer_text], 'answer_start': [start_index]}
                    })
                    found_in_chunk_count += 1
                    break
        elif answer_tokens_len >= max_answer_len_tokens:
            # --- SCENARIO B: Long Answer ---
            long_answer_as_context = answer_text
            answer_spans = chunk_text_by_sentence(long_answer_as_context, tokenizer, chunk_size=150, overlap_sentences=1)
            for span in answer_spans:
                start_index = long_answer_as_context.find(span)
                if start_index != -1:
                    training_data.append({
                        'context': long_answer_as_context, 'question': question,
                        'answers': {'text': [span], 'answer_start': [start_index]}
                    })

    print(f"\nProcessing completed. Generated {len(training_data)} training examples.")
    print(f"For Scenario A (short answers), {found_in_chunk_count} answers were found in a chunk.")
    return unique_chunks, training_data


In [44]:

DATASET_PATH = '../data/intern_screening_dataset.csv'  # SET THE PATH
MODEL_CHECKPOINT = 'distilbert-base-uncased'
SQUAD_DATA_PATH = '../data/squad_training_data_fixed.json'

tokenizer = AutoTokenizer.from_pretrained(MODEL_CHECKPOINT)
df_original = pd.read_csv(DATASET_PATH).dropna().sample(n=1000, random_state=42)  # Using a sample for faster execution

unique_chunks, squad_data = create_squad_dataset_pipeline(df_original, tokenizer)

print("\n--- Example of Generated Training Data ---")
if squad_data:
    print(json.dumps(squad_data[0], indent=2, ensure_ascii=False))
    
    with open(SQUAD_DATA_PATH, 'w', encoding='utf-8') as f:
        json.dump(squad_data, f, ensure_ascii=False, indent=4)
    print(f"\nTraining dataset saved to '{SQUAD_DATA_PATH}'")

Step 1: Aggregating answers to create long contexts...
Step 2: Applying sentence-based chunking to long contexts...


100%|██████████| 992/992 [00:00<00:00, 1276.03it/s]


Knowledge base created with 1201 unique chunks.

Step 3: Generating training data from 1000 original rows...


100%|██████████| 1000/1000 [00:01<00:00, 954.60it/s]


Processing completed. Generated 904 training examples.
For Scenario A (short answers), 529 answers were found in a chunk.

--- Example of Generated Training Data ---
{
  "context": "Chilaiditi syndrome is a medical condition in which a portion of the colon is abnormally positioned between the liver and the diaphragm. Symptoms vary, but may include abdominal pain, nausea, vomiting, and small bowel obstruction. In many cases, there are no symptoms and the interposition is an incidental finding. When no symptoms are present, the clinical finding is called Chilaiditi's sign.. The underlying cause of Chilaiditi syndrome is unknown. Treatment is symptomatic and supportive.",
  "question": "What is (are) Chilaiditi syndrome ?",
  "answers": {
    "text": [
      "Chilaiditi syndrome is a medical condition in which a portion of the colon is abnormally positioned between the liver and the diaphragm. Symptoms vary, but may include abdominal pain, nausea, vomiting, and small bowel obstruction. I